## Urban Graph Networks

author: Stephan Hausberg

This notebook has been created by Stephan Hausberg. It provides code snippets that let you walk through getting data from OpenStreetMap, handling this data to create maps with folium as well as solving problems from graph theory such as shortest path with Dijkstra's algorithm and Traveling Salesman Problem. Enjoy.

In [ ]:
# optional installation of missing libraries
%pip install overpy
%pip install folium

In [ ]:
import overpy
import folium as fl
from typing import Tuple, Dict, List
from dataclasses import dataclass

# Type definitions for better code clarity
BoundingBox = Tuple[float, float, float, float]
Coordinates = Tuple[float, float]

@dataclass
class MapConfig:
    """Configuration settings for the map visualization"""
    start_point: Coordinates = (52.517, 13.407)
    # Greater Berlin Mitte area
    bounding_box: BoundingBox = (52.489, 13.351, 52.551, 13.458)
    zoom_level: int = 14
    base_tile: str = "cartodbpositron"

def create_overpass_query(bbox: BoundingBox) -> str:
    """
    Creates an Overpass API query string for highways within a bounding box
    Args:
        bbox: Tuple of (south, west, north, east) coordinates
    Returns:
        Formatted query string
    """
    return (
        '[out:json][timeout:200];'
        f'(way["highway"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}););'
        'out body;>;out skel qt;'
    )

def fetch_highway_data(bbox: BoundingBox) -> overpy.Result:
    """
    Fetches highway data from Overpass API
    Args:
        bbox: Bounding box coordinates
    Returns:
        Overpass API result object
    """
    api = overpy.Overpass()
    query = create_overpass_query(bbox)
    return api.query(query)

def process_highway_data(result: overpy.Result) -> Tuple[Dict, Dict]:
    """
    Processes raw API result into structured dictionaries
    Args:
        result: Overpass API result
    Returns:
        Tuple of (highway_dict, nodes_dict)
    """
    highway_dict = {}
    nodes_dict = {}
    
    for way in result.ways:
        coordinates = []
        for node in way.nodes:
            lat, lon = float(node.lat), float(node.lon)
            coordinates.append([lat, lon])
            nodes_dict[node.id] = [lat, lon]
        highway_dict[way.id] = coordinates
    print('Highways haben: ' + str(len(highway_dict)))
    print('nodes haben: ' + str(len(nodes_dict)))
    return highway_dict, nodes_dict

def create_map(config: MapConfig, highway_dict: Dict, nodes_dict: Dict) -> fl.Map:
    """
    Creates a Folium map with highway and node layers
    Args:
        config: Map configuration settings
        highway_dict: Dictionary of highway coordinates
        nodes_dict: Dictionary of node coordinates
    Returns:
        Folium map object
    """
    # Initialize base map
    m = fl.Map(
        location=config.start_point,
        zoom_start=config.zoom_level,
        tiles=config.base_tile
    )

    # Add highways layer
    highways = fl.FeatureGroup("Streets", show=False)
    for coordinates in highway_dict.values():
        fl.PolyLine(
            coordinates,
            opacity=0.50,
            weight=2,
            color="grey"
        ).add_to(highways)
    highways.add_to(m)

    fl.LayerControl().add_to(m)
    return m

config = MapConfig()

# Fetch and process data
result = fetch_highway_data(config.bounding_box)
highway_dict, nodes_dict = process_highway_data(result)

# Create and return map
map_object = create_map(config, highway_dict, nodes_dict)


In [ ]:
map_object

In [ ]:
import networkx as nx
import numpy as np
from pyproj import Transformer

@dataclass
class BerlinPOI:
    """Berlin Points of Interest with their OSM node IDs"""
    BRANDENBURGER_TOR: int = 10719425945
    ALEXANDERPLATZ: int = 9213873349
    BODE_MUSEUM: int = 1217651537
    REICHSTAG: int = 1073123706
    BUNDESKANZLERAMT: int = 10298357735
    CHECKPOINT_CHARLIE: int = 10287452210

def create_street_graph(osm_ways, crs_in: int = 4326, crs_out: int = 25832) -> nx.Graph:
    """
    Creates a NetworkX graph from OSM way data with UTM coordinates for accurate distance calculation
    
    Args:
        osm_ways: OSM way objects from Overpass API
        crs_in: Input coordinate system (default: WGS84)
        crs_out: Output coordinate system (default: ETRS89 / UTM zone 32N)
    """
    transformer = Transformer.from_crs(crs_in, crs_out)
    G = nx.Graph()
    
    # Process each way and its nodes
    for way in osm_ways:
        nodes = way.nodes
        for i in range(len(nodes) - 1):
            # Transform coordinates to UTM for accurate distance calculation
            start_utm = transformer.transform(float(nodes[i].lat), float(nodes[i].lon))
            end_utm = transformer.transform(float(nodes[i+1].lat), float(nodes[i+1].lon))
            
            # Calculate Euclidean distance in meters
            distance = np.sqrt((end_utm[0] - start_utm[0])**2 + 
                             (end_utm[1] - start_utm[1])**2)
            
            # Add edge with distance as weight
            G.add_edge(nodes[i].id, nodes[i+1].id, weight=distance)
            
            # Store coordinates in original WGS84 for visualization
            G.nodes[nodes[i].id]['coords'] = (float(nodes[i].lat), float(nodes[i].lon))
            G.nodes[nodes[i+1].id]['coords'] = (float(nodes[i+1].lat), float(nodes[i+1].lon))
            
    return G

def visualize_route(G: nx.Graph, start_id: int, end_id: int, 
                   route_name: str = "Route", color: str = "blue") -> fl.FeatureGroup:
    """
    Calculates and visualizes shortest path using Dijkstra's algorithm
    """
    # Calculate shortest path
    try:
        path = nx.dijkstra_path(G, start_id, end_id, weight='weight')
        distance = nx.dijkstra_path_length(G, start_id, end_id, weight='weight')
        
        # Create route layer
        route_layer = fl.FeatureGroup(f"{route_name} ({distance:.0f}m)", show=True)
        
        # Create route coordinates
        route_coords = [G.nodes[node_id]['coords'] for node_id in path]
        
        # Add route line
        fl.PolyLine(
            route_coords,
            color=color,
            weight=4,
            opacity=0.8
        ).add_to(route_layer)
        
        # Add markers for start and end
        fl.Marker(
            G.nodes[start_id]['coords'],
            popup='Start',
            icon=fl.Icon(color='green')
        ).add_to(route_layer)
        
        fl.Marker(
            G.nodes[end_id]['coords'],
            popup='End',
            icon=fl.Icon(color='red')
        ).add_to(route_layer)
        
        return route_layer
        
    except nx.NetworkXNoPath:
        print(f"No path found between nodes {start_id} and {end_id}")
        return None

def solve_tsp_tour(G: nx.Graph, poi_nodes: List[int]) -> Tuple[List[int], float]:
    """
    Solves the Traveling Salesman Problem for given points of interest
    
    Returns:
        Tuple of (ordered tour node IDs, total distance)
    """
    # Create distance matrix
    n = len(poi_nodes)
    distances = np.zeros((n, n))
    
    # Calculate shortest path distances between all POI pairs
    for i in range(n):
        for j in range(i + 1, n):
            try:
                dist = nx.dijkstra_path_length(G, poi_nodes[i], poi_nodes[j], weight='weight')
                distances[i][j] = distances[j][i] = dist
            except nx.NetworkXNoPath:
                distances[i][j] = distances[j][i] = np.inf
    
    # Create TSP graph and solve
    tsp_graph = nx.from_numpy_array(distances)
    tour_indices = nx.approximation.traveling_salesman_problem(tsp_graph, cycle=True)
    
    # Convert indices back to node IDs
    tour = [poi_nodes[i] for i in tour_indices]
    total_distance = sum(nx.dijkstra_path_length(G, tour[i], tour[(i+1)%n], weight='weight')
                        for i in range(n))
    
    return tour, total_distance

def visualize_berlin_tour(G: nx.Graph, poi_names: Dict[int, str]) -> fl.Map:
    """
    Creates and visualizes a tour through Berlin's main attractions
    """
    # Initialize map centered on Brandenburg Gate
    m = fl.Map(
        location=G.nodes[BerlinPOI.BRANDENBURGER_TOR]['coords'],
        zoom_start=13,
        tiles="cartodbpositron"
    )
    
    # Get tour through POIs
    poi_nodes = list(poi_names.keys())
    tour, total_distance = solve_tsp_tour(G, poi_nodes)
    
    # Visualize tour segments
    for i in range(len(tour)):
        start = tour[i]
        end = tour[(i+1) % len(tour)]
        
        route_layer = visualize_route(
            G, start, end,
            route_name=f"Segment {i+1}: {poi_names[start]} → {poi_names[end]}",
            color='#505E96'
        )
        if route_layer:
            route_layer.add_to(m)
    
    fl.LayerControl().add_to(m)
    print(f"Total tour distance: {total_distance:.0f} meters")
    return m

# Usage example:
poi_names = {
    BerlinPOI.BRANDENBURGER_TOR: "Brandenburg Gate",
    BerlinPOI.ALEXANDERPLATZ: "Alexanderplatz",
    BerlinPOI.BODE_MUSEUM: "Bode Museum",
    BerlinPOI.REICHSTAG: "Reichstag",
    BerlinPOI.BUNDESKANZLERAMT: "Chancellery",
    BerlinPOI.CHECKPOINT_CHARLIE: "Checkpoint Charlie"
}

# Create graph from OSM data
G = create_street_graph(result.ways)

# Create single route example (Brandenburger Tor to Alexanderplatz)
m_single = fl.Map(location=G.nodes[BerlinPOI.BRANDENBURGER_TOR]['coords'],
                  zoom_start=13,
                  tiles="cartodbpositron")
route_layer = visualize_route(G, BerlinPOI.BUNDESKANZLERAMT, BerlinPOI.ALEXANDERPLATZ)
route_layer.add_to(m_single)
fl.LayerControl().add_to(m_single)

# Create full Berlin tour
m_tour = visualize_berlin_tour(G, poi_names)

In [ ]:
m_single